**Summary**

We did our entire project on Azure Databricks. This project can be run only on that platform with the required data. 

**Data Download**


The data was downloaded from https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr.


*   50 million flights
*   20 airlines

*   22 gigabytes
*   Duration : Jan 2012 - Dec 2019





**Data Merging** 

We then concatenated all the files under a folder called Data in azure cluster VM terminal. We used this command to concatenate csv files to a single csv. The output file is of 22 GB 

 *(select headers from first file and omit header from other files)*

awk 'FNR==1 && NR!=1{next;}{print}' file1.csv file2.csv ... file_n.csv > output.csv

**Uploading Data to Azure Blob Storage**

1. Created new container in Azure blob storage
2. Uploaded the entire concatenated data file ~ time taken 60 minutes.









We then created the azure data bricks workspace and imported the data from azure blob storage. After the import, we created a table using this data in azure databricks database.

We made the data available in databricks environment for creating our models

## We created cluster of 8 vCPU cores and configured **spark** in the cluster. We can read the data from the table as a spark data frame


In this project we look at the major airports in the United States with their flight delay. Flight delays inconvenience the passengers in running their schedules efficiently, especially those in the business sector, entrepreneurs, etc.Our goal is to use exploratory analysis and build a machine learning model to predict airline arrival delay. 

We addressed the following questions:
1. How does weather predict plane delays
2. What is the optimal time of day of the week or time of the years that is best to fly with minimal delays
3. Which flight carrier is prone to delays

**How does weather predict plane delays**

The first question we wanted to address was to see if we can predict the arrival delay given the flight had a departure delay due to weather. To address this question, we wanted  to first categorize this as a regression learning task. The target column was the arrival delay in minutes. We selected features like City, State, Destination, Distance, Origin Airport and Delay with Weather. There were certain string types which we converted to numerics with the String Indexer. This helped us to create a model to predict the arrival delays and the model had an accuracy of 95%. With this model we can predict the arrival delay given the flight had a departure delay due to weather.


In [ ]:
weatherDelayModel_results = weatherDelayModel.evaluate(test_data)
print(f'RSquared Error {weatherDelayModel_results.r2}\nAccuracy {weatherDelayModel_results.r2:.2%}')

RSquared Error 0.952493560905543
Accuracy 95.25%

(for full code, refer 1. How does weather predict plane delays.ipynb from results folder)

**What is the optimal time of day of the week or time of the years that is best to fly with minimal delays**

We then looked at  the flights that had a  minimal delay. The departure delay in minutes was the target column and it is a classification task. The features for this model were City, State, Destination, Distance, Origin Airport, Delay with weather and the string types were converted to numerics with string indexer. This model is used to predict the arrival delays and we achieved an AUC of 0.5. With this model, we could classify whether the given flight will have minimum flight delay or not for a particular departure origin airport.


In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='isMinimalDelay')
auc = evaluator.evaluate(pred_and_labels.predictions)
auc

Out[106]: 0.5

(for full code, refer 2. Optimal time that is best to fly with minimal delays.ipynbb from results folder)

**Predicting Delayed Flights**

The random forest model can predict/classify whether the given flight will have departure delays or not. We created the target column(is_delayed) using the departure delay in minutes column. If the departure delay is >15, the target column is set as 1 else 0. he features for this model were City, State, Destination, Distance, Origin Airport, Reporting airline and the string types were converted to numerics with string indexer.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator as BCE

print('AUC=',BCE(labelCol="isDelayed", metricName="areaUnderROC", rawPredictionCol='probability').evaluate(pred))

AUC= 0.8607323187703639

(for full code, refer 3. Predicting Delayed Flights from results folder)